# FPUS23 End-to-End Pipeline (Colab)
Runs dataset prep (CVAT→YOLO→COCO), verification, training, and evaluation.

In [ ]:
!nvidia-smi
!pip -q install ultralytics==8.* gdown lxml pillow opencv-python-headless matplotlib tqdm pycocotools seaborn

In [ ]:
# Download dataset ZIP from Google Drive and unzip
!gdown --id '1LL-r2hNiP6C190UBSE4v1FFCF3OQT9N3' -O /content/FPUS23_Dataset.zip
!unzip -q -o /content/FPUS23_Dataset.zip -d /content/
!ls -la /content | sed -n '1,120p'

In [ ]:
# Clone the repository containing scripts
!git clone https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git /content/repo
%cd /content/repo/scripts
!ls -la

In [ ]:
# Prepare dataset (writes YOLO and COCO formats)
!python prepare_fpus23.py \
+  --dataset-root "/content/FPUS23_Dataset" \
+  --project-root /content/fpus23_complete_project \
+  --group-split 1 \
+  --dry-run 0
!ls -la /content/fpus23_complete_project/dataset/fpus23_yolo

In [ ]:
# Verify dataset: counts + overlay sanity
%cd /content/repo/tools
!python verify_yolo_dataset.py \
+  --data /content/fpus23_complete_project/dataset/fpus23_yolo/data.yaml \
+  --split val \
+  --vis-out /content/tmp/vis_val \
+  --limit 24
from glob import glob
from IPython.display import Image, display
for p in sorted(glob('/content/tmp/vis_val/*.png'))[:12]:
    display(Image(p))

In [ ]:
# Sanity training (10 epochs)
%cd /content/repo/scripts
!python train_yolo_fpus23.py \
+  --data /content/fpus23_complete_project/dataset/fpus23_yolo/data.yaml \
+  --model yolo11s.pt \
+  --epochs 10 \
+  --batch 16 \
+  --imgsz 896 \
+  --lr 0.01 \
+  --cls 0.8 \
+  --rect \
+  --workers 2 \
+  --despeckle

In [ ]:
# Full training (100 epochs)
!python train_yolo_fpus23.py \
+  --data /content/fpus23_complete_project/dataset/fpus23_yolo/data.yaml \
+  --model yolo11s.pt \
+  --epochs 100 \
+  --batch 16 \
+  --imgsz 896 \
+  --lr 0.01 \
+  --cls 0.8 \
+  --rect \
+  --workers 2 \
+  --despeckle

In [ ]:
# Evaluate on test split
%cd /content/repo/scripts
!python eval_yolo_fpus23.py \
+  --weights runs/detect/fpus23/weights/best.pt \
+  --data /content/fpus23_complete_project/dataset/fpus23_yolo/data.yaml \
+  --split test \
+  --save-dir /content/results/yolo11s
!ls -la /content/results/yolo11s || true